In [1]:
#django env setup
import os, sys
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "intelliwiz_config,settings.py")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django
django.setup()

In [2]:
# Model imports
from apps.activity.models import Asset,AssetLog, Attachment, Device, DeviceEventlog, Event, Job, Jobneed, JobneedDetails, Location, Question, QuestionSet, QuestionSetBelonging, WorkPermit
from apps.attendance.models import PeopleEventlog, TestGeo, Tracking
from apps.onboarding.models import Bt,  GeofenceMaster, Shift, TypeAssist
from apps.peoples.models import Capability, People, PermissionGroup, Pgbelonging, Pgroup
from apps.reminder.models import Reminder
from apps.tenants.models import Tenant
from apps.y_helpdesk.models import EscalationMatrix, Ticket
from apps.core import utils
import pandas as pd

# Shell Plus Django Imports
from django.db import transaction
from django.db.models import Avg, Case, Count, F, Max, Min, Prefetch, Q, Sum, When, ExpressionWrapper, Value as V
from django.db.models.functions import Concat, Cast
from django.utils import timezone
from django.urls import reverse
from django.db.models import Exists, OuterRef, Subquery, FloatField, CharField, IntegerField
from datetime import datetime, timedelta

In [3]:
import pandas as pd
from IPython.display import display, clear_output
import ipywidgets as widgets
from IPython.display import display as ipy_display

def dq(queryset):
    if not queryset: return None
    if queryset := list(queryset):
    
        if isinstance(queryset, (list, tuple)):
            # Handle values_list() queryset
            df = pd.DataFrame(queryset)
        elif isinstance(queryset, dict):
            # Handle values() queryset
            df = pd.DataFrame.from_records([queryset])
        else:
            # Handle regular queryset
            data = list(queryset.values())
            df = pd.DataFrame(data)

        # Set pandas display options to show all rows and columns
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', None)

        # Get the specified column names
        columns = list(queryset[0].keys())

        # Filter the DataFrame based on the specified columns
        df_filtered = df[columns]

        # Create a scrollable output widget
        output_widget = widgets.Output()

        # Display the DataFrame inside the output widget
        with output_widget:
            clear_output()
            display(df_filtered)

        # Display the output widget
        ipy_display(output_widget)
    else:
        print("Empty Queryset")

(37, {'onboarding.TypeAssist': 37})

In [10]:
qset = TypeAssist.objects.values().order_by('-id')
dq(qset)

Output()

In [3]:
l.asset_set.annotate

NameError: name 'l' is not defined

In [4]:
Pgbelonging.objects.get_assigned_sites_to_people(13, False, True)

<QuerySet [{'buname': 'Developer Site', 'bucode': 'DEVSITE', 'butype_id': None, 'enable': True, 'cdtz': datetime.datetime(2023, 6, 25, 20, 39, 14, tzinfo=<UTC>), 'mdtz': datetime.datetime(2023, 6, 25, 20, 39, 14, tzinfo=<UTC>), 'siteincharge_id': None, 'cuser_id': 2, 'muser_id': 2, 'skipsiteaudit': False, 'identifier_id': 24, 'bupreferences': {'tag': '', 'address': '', 'validip': '', 'maxadmins': 5, 'validimei': '', 'usereliver': False, 'controlroom': [], 'guardstrenth': 0, 'malestrength': 0, 'pvideolength': 10, 'siteopentime': '', 'siteclosetime': '', 'webcapability': [], 'femalestrength': 0, 'ispermitneeded': False, 'mobilecapability': [], 'reportcapability': [], 'portletcapability': [], 'permissibledistance': 0, 'reliveronpeoplecount': 0, 'nearbyemergencycontacts': []}, 'bu_id': 9}]>

In [7]:
PeopleEventlog.objects.filter(peventtype__tacode='MARK').order_by('-cdtz').values()[0]

{'id': 415,
 'tenant_id': 1,
 'cuser_id': 356,
 'muser_id': 356,
 'cdtz': datetime.datetime(2023, 7, 4, 12, 23, 20, tzinfo=<UTC>),
 'mdtz': datetime.datetime(2023, 7, 4, 12, 23, 29, tzinfo=<UTC>),
 'ctzoffset': 330,
 'uuid': UUID('a616d57e-bde3-1c04-9234-0123456789ab'),
 'people_id': 334,
 'client_id': 13,
 'bu_id': 14,
 'shift_id': 1,
 'verifiedby_id': 1,
 'geofence_id': 1,
 'peventtype_id': 68,
 'transportmodes': ['NONE'],
 'punchintime': datetime.datetime(2023, 7, 4, 12, 23, 20, tzinfo=<UTC>),
 'punchouttime': datetime.datetime(2023, 7, 4, 12, 23, 29, tzinfo=<UTC>),
 'datefor': datetime.date(2023, 7, 4),
 'distance': 0.0,
 'duration': 0,
 'expamt': 0.0,
 'accuracy': 72.9000015258789,
 'deviceid': '562817f5d303de0c',
 'startlocation': <Point object at 0x7fa6367d8b10>,
 'endlocation': <Point object at 0x7fa6367d8c10>,
 'journeypath': None,
 'remarks': 'None',
 'facerecognitionin': True,
 'facerecognitionout': True,
 'peventlogextras': {'model': 'VGG',
  'threshold': '0.4',
  'distance

In [8]:
Ticket.objects.filter(id=36).delete()

(1, {'y_helpdesk.Ticket': 1})

In [24]:
from django.db.models import Q, F, ExpressionWrapper, fields
from django.utils import timezone
from django.db import connection

def fun_getexttourjobneed(_peopleid, _clientid):
    groupids = Pgbelonging.objects.filter(people_id=_peopleid).exclude(pgroup_id=-1).values_list('pgroup_id', flat=True)

    current_date = timezone.now()
    next_day = current_date + timezone.timedelta(days=1)

    job_needs = Jobneed.objects.filter(
        Q(Q(plandatetime__date__range=[current_date.date(), next_day.date()]) | Q(plandatetime__lte=current_date, expirydatetime__gte=current_date)),
        Q(people_id=_peopleid) | Q(cuser_id=_peopleid) | Q(muser_id=_peopleid) | Q(pgroup_id__in=groupids),
        client_id=_clientid,
        identifier='EXTERNALTOUR'
        
    ).values(
        'id', 'jobdesc', 'plandatetime', 'expirydatetime', 'gracetime', 'receivedonserver', 'starttime', 'endtime', 
        'gpslocation', 'remarks', 'cdtz', 'mdtz', 'pgroup_id', 'asset_id', 'cuser_id', 'frequency', 'job_id', 'jobstatus', 
        'jobtype', 'muser_id', 'performedby_id', 'priority', 'qset_id', 'scantype', 'people_id', 'attachmentcount', 
        'identifier', 'parent_id', 'bu_id', 'client_id', 'seqno', 'ticketcategory_id', 'ctzoffset', 'multifactor', 'uuid'
    ).annotate(
        adjusted_plan_date=ExpressionWrapper(F('plandatetime') + F('ctzoffset') * timezone.timedelta(minutes=1), output_field=fields.DateTimeField())
    )

    return job_needs


In [26]:
fun_getexttourjobneed(372, 14)

<QuerySet []>

In [4]:
# Get the QuestionSets with related questions
questionsets_with_questions = QuestionSet.objects.annotate(
    num_questions=Count('questionsetbelonging')
).filter(num_questions__gt=0)

In [6]:
questionsets_with_questions.filter(qsetname = 'TLB Inspection Check')

<QuerySet []>